In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/Louis/ml_projects/ToxPost'

In [ ]:
import src.features.preprocess as preprocess

..loading the data..


  0%|          | 81/159570 [00:00<03:28, 765.21it/s]

..using NLP to clean each comment in the corpus.. 


100%|██████████| 159570/159570 [02:45<00:00, 966.02it/s] 


..writing the results to the cleaned data file
..using TfIdf to shrink each comment in the corpus to size 100..


...

..loading the Glove embedding..


1193515it [00:18, 62906.18it/s]


using PCA to reduce the embedding space of each word to size 20


100%|██████████| 159570/159570 [00:07<00:00, 20066.01it/s]


..writing the results to the embedded data file..
